In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import imdb
import seaborn as sns
import matplotlib.pyplot as plt
import re
from datetime import datetime
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset

from model.RecommendationModel import RecommendationModel
from dataloader.RecommendationDataset import RecommendationDataset

In [2]:
features = ['product', 'year', 'Documentary', 'Western', 'Mystery', 'IMAX', 'Drama', 'Biography', 'Film-Noir', 
            'Music', 'Short', 'Thriller', 'Sport', 'Fantasy', 'Family', 'Children', 'Crime', 'Horror', 'Adult', 
            'Animation', 'Comedy', 'History', 'Adventure', 'Romance', 'War', 'Musical', 'Sci-Fi', 'Action']
contexts = ['daytime', 'weekend']

batch_size = 5120

train_data = RecommendationDataset(dataset_type='train',
                                   read_type='pkl',
                                   data_path='data/full_dataset.csv', 
                                   features=features, 
                                   contexts=contexts)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = RecommendationDataset(dataset_type='test',
                                  read_type='pkl',
                                  data_path='data/full_dataset.csv', 
                                  features=features, 
                                  contexts=contexts)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [3]:
model = RecommendationModel(lr=0.1, 
                            users_count=max(train_data.all_users)+1, 
                            movies_count=max(train_data.all_movies)+1, 
                            embedding_size=50, 
                            features_count=len(features), 
                            context_count=len(contexts))

trainer = pl.Trainer(max_epochs=100, accelerator='gpu', devices=1)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type      | Params
----------------------------------------------
0 | loss_fn         | MSELoss   | 0     
1 | user_embedding  | Embedding | 14.2 M
2 | movie_embedding | Embedding | 2.9 M 
3 | um_dense        | Linear    | 5.0 K 
4 | um_dense_2      | Linear    | 510   
5 | movie_features  | Linear    | 290   
6 | user_context    | Linear    | 6     
7 | final_fc1       | Linear    | 230   
8 | output          | Linear    | 11    
----------------------------------------------
17.1 M    Trainable params
0         Non-trainable params
17.1 M    Total params
68.290    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/swynimko/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/swynimko/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/home/swynimko/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
